In [ ]:
import psycopg2
import pandas as pd

In [ ]:
def create_dbs():
    """
        Create the datawarehouse and codeopfintech databases.
        If any of them already exists, Drop and create the database.
    Returns:
        success(boolean): wether the operation succeed or not.
    """
    try:
        # connect to default database
        conn = psycopg2.connect("postgresql://postgres:codeop@db")
        conn.set_session(autocommit=True)
        cur = conn.cursor()

        # create databases with UTF8 encoding
        cur.execute("DROP DATABASE IF EXISTS datawarehouse")
        cur.execute("DROP DATABASE IF EXISTS codeopfintech")
        cur.execute(
            "CREATE DATABASE datawarehouse WITH ENCODING 'utf8' TEMPLATE template0"
        )
        cur.execute(
            "CREATE DATABASE codeopfintech WITH ENCODING 'utf8' TEMPLATE template0"
        )

        # close connection
        conn.close()

        # connect to data_warehouse database
        # conn = psycopg2.connect("postgresql://postgres:letmein@db/datawarehouse")
        # cur = conn.cursor()
        return True
    except (Exception, psycopg2.Error) as error:
        print("Failed to creating dbs: ", error)
        return False

In [ ]:
def get_db(db_name):
    """
    Establish connection to a database and return its connection.
    Args:
        dn_name (str): The name of the database you want to connect
    Returns:
        cursor(psycopg2.cursor): The psycopg2 cursor
        connection(psycopg2.connection): The db connection
    """
    try:
        conn = psycopg2.connect(f"postgresql://postgres:codeop@db/{db_name}")

        return conn.cursor(), conn
    except (Exception, psycopg2.Error) as error:
        print("Failed to getting db: ", error)
        return False, False

In [ ]:
create_dbs()

In [ ]:
create_table = ("""
    CREATE TABLE cards (
      id varchar(34),
      user_id int,
      iban varchar(34),
      pan varchar(255),
      pin varchar(4),
      cvv varchar(255),
      track1 varchar(255),
      track2 varchar(255),
      expiring_date varchar(255),
      PRIMARY KEY (id)
    )
""")

In [ ]:
cards_table_insert = ("""
    INSERT INTO cards(
        id,
        user_id,
        iban,
        pan,
        pin,
        cvv,
        track1,
        track2,
        expiring_date)
    VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s)
""")

In [ ]:
cur, conn = get_db("datawarehouse")

In [ ]:
cur.execute(create_table)
conn.commit()

In [ ]:
df = pd.read_csv("credit_cards.csv")
df.head()

In [ ]:
cards_row = df[:].values[0]
cards_row

In [ ]:
cur.execute(cards_table_insert, cards_row)
conn.commit()

In [ ]:
conn.close()